<a href="https://colab.research.google.com/github/mobley-trent/ml-from-scratch/blob/mobley-trent-patch-1/decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A decision tree is a supervised learning algorithm that is used for classification and regression modeling. It is a tree-structured classifier, where internal nodes represent the features of a dataset, branches represent the decision rules and each leaf node represents the outcome. It is one of the most widely used and practical methods for supervised learning.

Decision trees are constructed via an algorithmic approach that identifies ways to split a data set based on various conditions. There are two types of decision trees: classification trees and regression trees. Classification trees are used when the target variable is categorical, while regression trees are used when the target variable is continuous.

In [ ]:
import numpy as np
from collections import Counter


class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

The `Node` class represents a node in the decision tree, which has a feature and threshold to split the data, as well as left and right child nodes. It also has a value that is assigned when the node is a leaf node. The `is_leaf_node()` method checks whether the node is a leaf node or not.

The DecisionTree class has the following attributes:

- `min_samples_split`: the minimum number of samples required to split an internal node.
- `max_depth`: the maximum depth of the tree.
- `n_features`: the number of features to consider when looking for the best split. If `None`, then all features are considered.
- `root`: the root node of the decision tree.

The `fit()` method takes the input data X and corresponding labels y and grows the decision tree using the `_grow_tree()` method.

The `_grow_tree()` method recursively grows the decision tree. It checks if the stopping criteria is met or not. If met, it assigns a leaf value to the node. Otherwise, it randomly selects a subset of features, finds the best split using the `_best_split()` method, creates child nodes using the `_split()` method, and grows the left and right sub-trees recursively.

The `_best_split()` method finds the feature and threshold that results in the highest information gain using the `_information_gain()` method. It iterates through all features and their unique thresholds to find the best split.

The `_information_gain()` method calculates the information gain of a split by calculating the **entropy** of the parent and child nodes. It calculates the entropy of the left and right child nodes using the `_entropy()` method and returns the information gain.

The `_split()` method splits the data into left and right indices based on the feature threshold.

The `_entropy()` method calculates the entropy of the input labels y.

The `_most_common_label()` method returns the most common label in the input labels y.

The `predict()` method takes input data X and returns the predicted labels by traversing the decision tree using the `_traverse_tree()` method.

The `_traverse_tree()` method traverses the decision tree recursively by checking the node feature and threshold and moving down to the left or right child node until it reaches a leaf node, which returns the assigned value.

In [ ]:
# Testing the algorithm with data from sklearn.datasets

from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)

In [ ]:
clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

In [ ]:
def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

acc = accuracy(y_test, predictions)
print(acc)

0.9210526315789473
